In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import json
from apify_client import ApifyClient

# Configuración de paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Importar configuración
from config import APIFY_API_KEY, DB_PATH
from src.utils.database import get_db_connection, insert_dataframe_to_table

# Configuración de Apify
client = ApifyClient(APIFY_API_KEY)

# URLs de búsqueda (vamos con calma, solo 2 primero)
SEARCH_URLS = [
    #"https://www.tripadvisor.es/Restaurants-g187499-oa120-Girona_Province_of_Girona_Catalonia.html", 
    #"https://www.tripadvisor.es/Restaurants-g187499-oa150-Girona_Province_of_Girona_Catalonia.html",
    #"https://www.tripadvisor.es/Restaurants-g187499-oa180-Girona_Province_of_Girona_Catalonia.html",
]

In [3]:
def run_apify_actor_slow():
    """Ejecuta el actor de Apify con configuración ultra-lenta para evitar bloqueos"""
    print("🚀 Ejecutando actor de Apify (modo lento)...")
    print(f"📋 Número de URLs: {len(SEARCH_URLS)}")
    
    try:
        run = client.actor("maxcopell/tripadvisor").call(
            run_input={
                "startUrls": [{"url": url} for url in SEARCH_URLS],
                "maxItems": 20,
                "proxyConfiguration": {
                    "useApifyProxy": True,
                    "apifyProxyGroups": ["RESIDENTIAL"],
                    "apifyProxyCountry": "ES"
                },
                "maxConcurrency": 1,
                "minTimeBetweenRequests": 10000,
                "searchLanguage": "es",
                "reviews": False,
            },
            wait_secs=600
        )
        
        print("📦 Recopilando datos...")
        dataset = client.dataset(run["defaultDatasetId"]).list_items().items
        
        if not dataset:
            print("❌ No se encontraron datos")
            return None
        
        print(f"✅ Se obtuvieron {len(dataset)} restaurantes")
        return dataset
        
    except Exception as e:
        print(f"❌ Error ejecutando el actor de Apify: {e}")
        return None

In [4]:
# Ejecutar el actor y obtener datos
dataset = run_apify_actor_slow()

🚀 Ejecutando actor de Apify (modo lento)...
📋 Número de URLs: 1


[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> Status: RUNNING, Message: 
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:30.550Z ACTOR: Pulling Docker image of build 0EU8Lvjqh6AkaDl65 from registry.
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:30.552Z ACTOR: Creating Docker container.
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:31.321Z ACTOR: Starting Docker container.
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:32.008Z Will run command: xvfb-run -a -s "-ac -screen 0 1920x1080x24+32 -nolisten tcp" /bin/sh -c npm run start:prod --silent
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:37.548Z INFO  System info {"apifyVersion":"3.1.15","apifyClientVersion":"2.8.4","crawleeVersion":"3.7.2","osType":"Linux","nodeVersion":"v20.19.4"}
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:37.710Z INFO  Input validation OK
[apify.tripadvisor runId:Z1RScmaoZMikYftYy] -> 2025-09-20T20:39:37.937Z 

📦 Recopilando datos...
✅ Se obtuvieron 20 restaurantes


In [5]:
def extract_url_path(full_url):
    """Extrae solo la parte del path de la URL"""
    if full_url and 'tripadvisor.com' in full_url:
        return full_url.split('tripadvisor.com')[-1]
    return full_url

def extract_cuisines(cuisines_data):
    """Convierte cocinas a string"""
    if not cuisines_data:
        return ''
    
    if isinstance(cuisines_data, list):
        if cuisines_data and isinstance(cuisines_data[0], dict):
            return ', '.join([c.get('name', '') for c in cuisines_data])
        return ', '.join(cuisines_data)
    return str(cuisines_data)

In [6]:
# Explorar los datos
print("🔍 EXPLORACIÓN DE DATOS OBTENIDOS")
print("="*50)

if dataset:
    # Crear DataFrame para exploración
    df_raw = pd.DataFrame(dataset)
    print(f"📊 Total de items: {len(df_raw)}")
    print(f"📋 Columnas: {list(df_raw.columns)}")
    
    # Mostrar primeras filas
    print("\n👀 Primeras filas:")
    display(df_raw.head())
    
    # Ver estructura del primer item
    print("\n📋 Estructura del primer restaurante:")
    first_item = dataset[0]
    for key, value in first_item.items():
        print(f"  {key}: {type(value).__name__} = {str(value)[:80]}...")
else:
    print("No hay datos para explorar")

🔍 EXPLORACIÓN DE DATOS OBTENIDOS
📊 Total de items: 20
📋 Columnas: ['reviewTags', 'id', 'type', 'category', 'subcategories', 'name', 'locationString', 'description', 'image', 'photoCount', 'rankingPosition', 'rating', 'rawRanking', 'phone', 'address', 'addressObj', 'localName', 'localAddress', 'localLangCode', 'email', 'latitude', 'longitude', 'webUrl', 'website', 'rankingString', 'rankingDenominator', 'rankingSource', 'neighborhoodLocations', 'nearestMetroStations', 'ancestorLocations', 'ratingHistogram', 'numberOfReviews', 'isClosed', 'isLongClosed', 'openNowText', 'priceLevel', 'dishes', 'hours', 'menuWebUrl', 'ownersTopReasons', 'isNearbyResult', 'photos', 'travelerChoiceAward', 'cuisines', 'dietaryRestrictions', 'establishmentTypes', 'features', 'mealTypes', 'isClaimedIcon', 'isClaimedText', 'orderOnline', 'input']

👀 Primeras filas:


,reviewTags,id,type,category,subcategories,name,locationString,description,image,photoCount,...,travelerChoiceAward,cuisines,dietaryRestrictions,establishmentTypes,features,mealTypes,isClaimedIcon,isClaimedText,orderOnline,input
0,[],24154892,RESTAURANT,restaurant,[],Corb Mari,"Girona, Province of Girona, Catalonia",None,https://media-cdn.tripadvisor.com/media/photo-...,1,...,None,"[Seafood, Mediterranean, Spanish, Healthy]",[],[Restaurants],[],"[Lunch, Dinner]",False,None,[],https://www.tripadvisor.es/Restaurants-g187499...
1,[],26271548,RESTAURANT,restaurant,[],Txots Girona,"Girona, Province of Girona, Catalonia",None,https://media-cdn.tripadvisor.com/media/photo-...,26,...,None,"[Spanish, Grill, Catalan]",[Vegetarian friendly],[Restaurants],"[Reservations, Seating, Table Service]","[Lunch, Dinner, Late Night, Drinks]",False,None,[],https://www.tripadvisor.es/Restaurants-g187499...
2,[],17712371,RESTAURANT,restaurant,[Café],Tandem Cafe Girona,"Girona, Province of Girona, Catalonia",None,https://media-cdn.tripadvisor.com/media/photo-...,23,...,None,"[Cafe, Seafood, Mediterranean, Spanish]",[],[Restaurants],"[Reservations, Seating, Wheelchair Accessible,...","[Breakfast, Lunch, Dinner, Brunch, Drinks]",False,None,[],https://www.tripadvisor.es/Restaurants-g187499...
3,[],12717475,RESTAURANT,restaurant,[],Blessing Restaurant,"Girona, Province of Girona, Catalonia",None,https://media-cdn.tripadvisor.com/media/photo-...,11,...,None,"[Latin, Spanish]",[],[Restaurants],"[Reservations, Seating, Wheelchair Accessible,...","[Lunch, Dinner, Brunch]",False,None,[],https://www.tripadvisor.es/Restaurants-g187499...
4,[],26769090,RESTAURANT,restaurant,[Sit down],Somia Vins I Llibres,"Girona, Province of Girona, Catalonia",None,https://media-cdn.tripadvisor.com/media/photo-...,5,...,None,"[Spanish, Fusion, Catalan]",[],[Restaurants],"[Reservations, Outdoor Seating, Seating, Stree...","[Breakfast, Dinner, Brunch, Drinks]",False,None,[],https://www.tripadvisor.es/Restaurants-g187499...



📋 Estructura del primer restaurante:
  reviewTags: list = []...
  id: str = 24154892...
  type: str = RESTAURANT...
  category: str = restaurant...
  subcategories: list = []...
  name: str = Corb Mari...
  locationString: str = Girona, Province of Girona, Catalonia...
  description: NoneType = None...
  image: str = https://media-cdn.tripadvisor.com/media/photo-m/1280/2c/d6/f8/87/caption.jpg...
  photoCount: int = 1...
  rankingPosition: int = 213...
  rating: int = 5...
  rawRanking: float = 3.143367290496826...
  phone: str = +34 621 33 67 69...
  address: str = Passeig del Molinet 7, L_Estartit Bar Corb Mari, 17258 Girona Spain...
  addressObj: dict = {'street1': 'Passeig del Molinet 7, L_Estartit', 'street2': 'Bar Corb Mari', 'ci...
  localName: str = Corb Mari...
  localAddress: NoneType = None...
  localLangCode: str = es...
  email: NoneType = None...
  latitude: float = 42.053364...
  longitude: float = 3.208051...
  webUrl: str = https://www.tripadvisor.com/Restaurant_Review

In [7]:
if dataset:
    # Procesar datos para la BD
    restaurantes_data = []
    
    for item in dataset:
        # Función de limpieza para rango_precio
        def clean_price_level(price_level):
            if price_level is None:
                return ''
            elif isinstance(price_level, (int, float)):
                return str(price_level)
            elif isinstance(price_level, str):
                return price_level
            else:
                return str(price_level)
        
        # Convertir listas a strings
        platos = item.get('dishes', '')
        if isinstance(platos, list):
            platos = ', '.join(platos)
        
        tipo_cocina = extract_cuisines(item.get('cuisines', []))
        if isinstance(tipo_cocina, list):
            tipo_cocina = ', '.join(tipo_cocina)
        
        restaurante = {
            'tripadvisor_id' : item.get('id', ''),
            'nombre' : item.get('name',''),
            'telefono' : item.get('phone',''),
            'direccion' : item.get('address',''),
            'localizacion' : item.get('locationString',''),
            'email' : item.get('email',''),
            'latitud' : item.get('latitude',''),
            'longitud' : item.get('longitude',''),
            'website' : extract_url_path(item.get('website','')),                        
            'rating' : item.get('rating',''),
            'ranking' : item.get('rankingPosition',''),
            'rango_precio': clean_price_level(item.get('priceLevel', '')),
            'platos' : platos,
            'tipo_cocina': tipo_cocina
        }
        restaurantes_data.append(restaurante)
    
    df_clean = pd.DataFrame(restaurantes_data)
    
    # Limpieza adicional por si acaso
    df_clean['rango_precio'] = df_clean['rango_precio'].fillna('').astype(str)
    
    print(f"✅ Datos procesados: {len(df_clean)} restaurantes")

✅ Datos procesados: 20 restaurantes


In [8]:
if 'df_clean' in locals() and not df_clean.empty:
    conn = None
    try:
        print("💾 Guardando en base de datos...")
        conn = get_db_connection()
        
        # VERIFICACIÓN simple
        cursor = conn.cursor()
        cursor.execute("PRAGMA database_list;")
        db_info = cursor.fetchall()
        print("📋 Base de datos conectada:", db_info[0][2])
        
        # Verificar si la tabla existe
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Restaurantes'")
        if cursor.fetchone():
            print("✅ Tabla Restaurantes encontrada")
            
            # Insertar datos
            rows_inserted = insert_dataframe_to_table(df_clean, 'Restaurantes', conn)
            conn.commit()
            print(f"✅ {rows_inserted} filas insertadas")
        else:
            print("❌ Tabla Restaurantes no existe")
        
        if conn:
            conn.close()
        
    except Exception as e:
        print(f"❌ Error guardando en BD: {e}")
        if conn:
            conn.rollback()
else:
    print("No hay datos limpios para guardar")

💾 Guardando en base de datos...
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📋 Base de datos conectada: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
✅ Tabla Restaurantes encontrada
✅ 20 filas insertadas en la tabla 'Restaurantes'.
✅ 20 filas insertadas
